<a href="https://colab.research.google.com/github/victormurcia/VCHAMPS/blob/main/VCHAMPS_Data_CleanUp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#General utilities
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm  # Import tqdm for the progress bar
import glob,shutil,os,warnings,math,time,sys,re
from typing import List
import dask.dataframe as dd

#For converting states to their abbreviations
#!pip install us
#import us

#For performing UTC normalization on datetime columns based on the STATE column
import pytz

#For Slider viz
import ipywidgets as widgets
from IPython.display import display, clear_output,HTML

#For EDA
#!pip install dataprep
#from dataprep.eda import create_report

#Enable data to be extracted and downloaded from my Google Drive
from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ICD Mapping Functions (Adi)
Adi wrote this routine to map the diagnosis columns into ICD Codes.

In [2]:
def read_icd_codes(path):
    icd_codes = pd.read_csv(path).drop('Unnamed: 0', axis = 1)
    return icd_codes

# Function to split semicolon separated diagnoses into a list of diagnoses
def split_diagnoses(diagnoses):
    return [dx.lower().replace(",", "").strip() for dx in diagnoses.split(';')]

def process_diagnoses_dataframes(dataframes,icd_codes):
    final_df_list = []
    for idx, df in enumerate(dataframes):
        # Get the columns containing diagnoses (columns with 'icd10' in their name)
        diagnosis_columns = [col for col in df.columns if 'icd10' in col.lower()]
        print(idx, diagnosis_columns)
        df.compute()

        # Create a new column with a single list of diagnoses
        for i in range(len(diagnosis_columns)):
            if i == 0:
              df['diagnosis'] = df[diagnosis_columns[i]].apply(split_diagnoses)
            else:
              df['diagnosis'] = df['diagnosis'] + df[diagnosis_columns[i]].apply(split_diagnoses)

        # Explode the columns of lists so each row has a single diagnosis
        df = df.explode('diagnosis')
        df.reset_index(drop = True)
        # Merge to icd_codes csv
        df = df.merge(icd_codes, left_on='diagnosis', right_on='description', how='left')
        df = df.drop(columns = diagnosis_columns + ['description'])
        final_df_list.append(df)
    return final_df_list

def process_diagnoses_dataframe(df, icd_codes):
    # Get the columns containing diagnoses (columns with 'icd10' in their name)
    diagnosis_columns = [col for col in df.columns if 'icd10' in col.lower()]
    print(diagnosis_columns)
    df.compute()

    # Create a new column with a single list of diagnoses
    for i in range(len(diagnosis_columns)):
        if i == 0:
            df['diagnosis'] = df[diagnosis_columns[i]].apply(split_diagnoses)
        else:
            df['diagnosis'] = df['diagnosis'] + df[diagnosis_columns[i]].apply(split_diagnoses)

    # Explode the columns of lists so each row has a single diagnosis
    df = df.explode('diagnosis')
    df.reset_index(drop=True)

    # Merge with icd_codes dataframe
    df = df.merge(icd_codes, left_on='diagnosis', right_on='description', how='left')
    df = df.drop(columns=diagnosis_columns + ['description'])

    return df

In [3]:
icd_codes = read_icd_codes('/content/drive/MyDrive/icd_codes_cc.csv')
icd_codes

,code,description,billable,cc Status
0,A00,cholera,0,NCC
1,A000,cholera due to vibrio cholerae 01 biovar cholerae,1,CC
2,A001,cholera due to vibrio cholerae 01 biovar eltor,1,CC
3,A009,cholera unspecified,1,CC
4,A01,typhoid and paratyphoid fevers,0,NCC
...,...,...,...,...
95697,U07,emergency use of u07,0,NCC
95698,U070,vaping-related disorder,1,NCC
95699,U071,covid-19,1,NCC
95700,U09,post covid-19 condition,0,NCC


# Cleaning Up The Data
I'm going to finish cleaning up the data since this is taking too long.

In [4]:
# Specify the path to the desired directory
directory_path = r'/content/drive/MyDrive/VCHAMPS - Train Data -  UTC'

# Change the current working directory to the desired directory
os.chdir(directory_path)

# Verify the current working directory
cwd = os.getcwd()

print(f"Current working directory: {cwd}")

Current working directory: /content/drive/MyDrive/VCHAMPS - Train Data -  UTC


In [5]:
def load_parquet_files(folder_path):
    # Get a list of all files in the current directory
    all_files = [f.path for f in os.scandir(folder_path) if f.is_file()]

    # Get a list of all subdirectories (dataset folders) in the current directory
    dataset_folders = [f.path for f in os.scandir(folder_path) if f.is_dir()]

    # Initialize an empty list to store the DataFrames
    dataframes = []

    # Load Parquet files in the current directory
    for file in all_files:
        if file.endswith('.parquet'):
            df = dd.read_parquet(file, engine='pyarrow')
            dataframes.append(df)

    # Load Parquet files in the subfolders
    for folder in dataset_folders:
        df = dd.read_parquet(folder + '/**/*.parquet', engine='pyarrow')
        dataframes.append(df)

    return dataframes

dataframes = load_parquet_files(cwd)

# Cleaning of Conditions DF
In this dataframe I'll convert the condition type into a categorical variable by one hot encoding it and converting the resulting columns into bools.

Once Adi gives me the mapping function, I'll convert those ICD10 codes and drop that column.

What do we want to with the 'Diagnosis sequence number or rank' column?

In [6]:
def preprocess_dataframe(df, col_name):
    # Categorize a column
    df[col_name] = df[col_name].astype('category')
    df[col_name] = df[col_name].cat.as_known()  # Ensure known categories

    # Get dummies for a specific column
    dummies = dd.get_dummies(df[col_name])

    # Convert dummy columns to bool data type
    for col in dummies.columns:
        dummies[col] = dummies[col].astype(bool)

    # Concatenate the dummies with the original DataFrame
    df_with_dummies = dd.concat([df, dummies], axis=1)

    # Drop a column
    new_df = df_with_dummies.drop(col_name, axis=1)

    # Clear df and dummies from memory
    del df, dummies

    return new_df

col_name = 'Condition type'
df = dataframes[0]
conditions_df = preprocess_dataframe(df,col_name)
#conditions_df.head()

/usr/local/lib/python3.10/dist-packages/dask/dataframe/multi.py:1269: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


In [7]:
conditions_df.dtypes

Internalpatientid                             int32
Age at condition documentation                 int8
Condition documented date            datetime64[ns]
Condition code icd10 subcategory             object
Diagnosis sequence number or rank            object
Diagnosis                                      bool
Problem                                        bool
dtype: object

In [6]:
def calculate_nan_percentage(df):
    # Calculate percentage of NaN values in each column
    nan_percentage = (df.isna().sum() / df.shape[0]) * 100
    return nan_percentage
#df = dataframes[0].compute()
#nan_percentage = calculate_nan_percentage(conditions_df)
#nan_percentage

About 4% of the entries in 'Diagnosis sequence number or rank' are NaNs. I'll drop them since there is no point on keeping them.

In [9]:
conditions_df = conditions_df.dropna(subset=['Diagnosis sequence number or rank'])
conditions_df = process_diagnoses_dataframe(conditions_df, icd_codes)
conditions_df = conditions_df.drop(columns=['diagnosis','billable'])
conditions_df = conditions_df.dropna(subset=['code','cc Status'])

['Condition code icd10 subcategory']


/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:4134: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Condition code icd10 subcategory', 'object'))

  warnings.warn(meta_warning(meta))


In [12]:
#nan_percentage = calculate_nan_percentage(conditions_df.compute())
#nan_percentage

Internalpatientid                    0.000000
Age at condition documentation       0.000000
Condition documented date            0.000000
Diagnosis sequence number or rank    0.000000
Diagnosis                            0.000000
Problem                              0.000000
code                                 3.264609
billable                             3.264609
cc Status                            3.264609
dtype: float64

In [10]:
# Save the Dask DataFrame as Parquet
conditions_df.to_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/conditions_train.parquet', engine='pyarrow')

Once Adi gives me the mapping function, I'll apply that to this dataframe. Whatever ICD codes are not mapped will be dropped. I'll also drop the 'Condition code icd10 subcategory' column at that point.

# Cleaning the Demographics Event DF
This dataframe will basically just need to be checked for NaNs and then one hot encode the Marital_Statis and Ruca_category columns

In [11]:
df = dataframes[1]
df.head()

,Internalpatientid,Age at update,Event date,Marital status,Ruca category
0,100028,79,2020-08-24 03:33:32,Married,Urban
1,100032,91,2008-02-07 05:03:27,Married,Rural
2,100046,73,2003-09-14 09:32:12,Married,Urban
3,100071,75,2019-04-25 08:33:42,Widowed,Urban
4,100091,81,2022-02-09 09:24:20,Never married,Urban


In [9]:
df = dataframes[1].compute()
nan_percentage = calculate_nan_percentage(df)
nan_percentage

Internalpatientid    0.0
Age at update        0.0
Event date           0.0
Marital status       0.0
Ruca category        0.0
dtype: float64

There are no NaNs in this dataframe. I'll convert the dataframe into categorical and move on.

In [12]:
df = dataframes[1]
dem_event_df = preprocess_dataframe(df,'Marital status')
dem_event_df = preprocess_dataframe(dem_event_df,'Ruca category')

/usr/local/lib/python3.10/dist-packages/dask/dataframe/multi.py:1269: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/dask/dataframe/multi.py:1269: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


In [11]:
dem_event_df.head()

/usr/local/lib/python3.10/dist-packages/dask/dataframe/multi.py:1269: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


,Internalpatientid,Age at update,Event date,Divorced,Married,Never married,Not specified (no value),Separated,Single,Unknown,Widowed,Highly rural,Not specified,Rural,Urban
0,100028,79,2020-08-24 03:33:32,False,True,False,False,False,False,False,False,False,False,False,True
1,100032,91,2008-02-07 05:03:27,False,True,False,False,False,False,False,False,False,False,True,False
2,100046,73,2003-09-14 09:32:12,False,True,False,False,False,False,False,False,False,False,False,True
3,100071,75,2019-04-25 08:33:42,False,False,False,False,False,False,False,True,False,False,False,True
4,100091,81,2022-02-09 09:24:20,False,False,True,False,False,False,False,False,False,False,False,True


In [13]:
dem_event_df = dem_event_df.drop(columns=['Not specified (no value)','Not specified'])

In [14]:
# Save the Dask DataFrame as Parquet
dem_event_df.to_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/demographics_event_df.parquet', engine='pyarrow')

# Cleaning Demographics Static DF
Check for NaNs and convert stuff to categorical.

In [12]:
df = dataframes[2].compute()
nan_percentage = calculate_nan_percentage(df)
nan_percentage

Internalpatientid    0.0
Ethnicity            0.0
Gender               0.0
Races                0.0
Veteran flag         0.0
dtype: float64

There are no NaNs in this dataframe. I'll convert the dataframe into categorical and move on.

In [15]:
dem_static_df = dataframes[2]

# Drop rows where 'Ethnicity' column contains 'Not specified'
dem_static_df = dem_static_df[dem_static_df['Ethnicity'] != 'Not specified']

dem_static_df = preprocess_dataframe(dem_static_df,'Gender')

# Drop rows where 'Races' column contains '(Censored)'
dem_static_df = dem_static_df[dem_static_df['Races'] != '(Censored)']

dem_static_df = preprocess_dataframe(dem_static_df,'Races')

# Define the value conversions
value_conversions = {'Yes': 1, 'No': 0}

# Convert specific values in the column
dem_static_df['Veteran flag'] = dem_static_df['Veteran flag'].map(value_conversions, na_action='ignore')

dem_static_df.head()

/usr/local/lib/python3.10/dist-packages/dask/dataframe/multi.py:1269: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/dask/dataframe/multi.py:1269: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


,Internalpatientid,Ethnicity,Veteran flag,Female,Male,Asian,Asian ; Other,Black or African American,Black or African American ; Asian,Black or African American ; Other,Black or African American ; White,Black or African American ; White ; Asian,Black or African American ; White ; Other,Not specified (no value),Other,White,White ; Asian,White ; Asian ; Other,White ; Other
0,168674,Hispanic or Latino,1,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,168681,Not Hispanic or Latino,1,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False
2,168696,Not Hispanic or Latino,1,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
3,168711,Not Hispanic or Latino,1,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False
4,168720,Not Hispanic or Latino,1,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [17]:
dem_static_df = dem_static_df.drop(columns=['Not specified (no value)'])

In [18]:
# Save the Dask DataFrame as Parquet
dem_static_df.to_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/demographics_static.parquet', engine='pyarrow')

# Cleaning Death DF

Check for NaNs

In [19]:
df = dataframes[3].compute()
nan_percentage = calculate_nan_percentage(df)
nan_percentage

Internalpatientid    0.0
Age at death         0.0
Death date           0.0
dtype: float64

In [20]:
# Save the Dask DataFrame as Parquet
df.to_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/death.parquet', engine='pyarrow')

No NaNs.

# Cleaning ED Visits DF

Check for NaNs and map diagnosis to ICD Codes

In [18]:
df = dataframes[4].compute()
nan_percentage = calculate_nan_percentage(df)
nan_percentage

Internalpatientid                            0.000000
Age at ed visit                              0.000000
Ed visit start date                          0.000000
Discharge date ed                            0.014186
Died during ed visit                         0.000000
First listed diagnosis icd10 subcategory     0.000000
Second listed diagnosis icd10 subcategory    0.000000
CV diagnosis                                 0.000000
dtype: float64

Hmmm, a small percentage of the Discharge dates are null. Does this represent a patient that is still currently hospitalized? Given that it is such a small percentage, I'm not going to worry about it for now and simply drop those observations.

In [21]:
ed_visits_df = dataframes[4]
# Drop rows with NaN or NaT values from a specific column
ed_visits_df = ed_visits_df.dropna(subset=['Discharge date ed'])

In [22]:
ed_visits_df = process_diagnoses_dataframe(ed_visits_df, icd_codes)
ed_visits_df = ed_visits_df.drop(columns=['diagnosis','billable'])
ed_visits_df = ed_visits_df.dropna(subset=['code','cc Status'])

['First listed diagnosis icd10 subcategory', 'Second listed diagnosis icd10 subcategory']


/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:4134: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('First listed diagnosis icd10 subcategory', 'object'))

  warnings.warn(meta_warning(meta))
/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:4134: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Secon

In [23]:
ed_visits_df.compute()

,Internalpatientid,Age at ed visit,Ed visit start date,Discharge date ed,Died during ed visit,CV diagnosis,code,cc Status
0,101689,64,2021-08-26 00:53:30,2021-08-26 04:24:27,No,1,I502,NCC
1,101689,64,2021-08-26 00:53:30,2021-08-26 04:24:27,No,1,N189,NCC
2,107210,71,2022-05-18 11:21:28,2022-05-18 18:30:21,No,0,A419,MCC
3,107210,71,2022-05-18 11:21:28,2022-05-18 18:30:21,No,0,I959,NCC
4,107866,61,2017-06-26 15:41:11,2017-06-26 23:33:50,No,0,R918,NCC
...,...,...,...,...,...,...,...,...
1099360,9883,90,2020-02-20 21:23:37,2020-02-20 22:56:30,No,0,W108,NCC
1099422,99235,87,2016-02-25 01:22:23,2016-02-25 06:33:45,No,0,R197,NCC
1099538,99902,66,2018-09-28 23:21:41,2018-09-28 23:56:55,No,0,M796,NCC
1099548,99934,84,2022-07-22 09:19:20,2022-07-22 09:42:57,No,0,M796,NCC


In [26]:
ed_visits_df['Died during ed visit'].value_counts().compute()

No     398447
Yes        68
Name: Died during ed visit, dtype: int64

In [27]:
# Assuming ed_visits_df is your Dask DataFrame
ed_visits_df['Died during ed visit'] = ed_visits_df['Died during ed visit'].map({'Yes': 1, 'No': 0}, meta=('Died during ed visit', 'int8'))

In [29]:
ed_visits_df.compute()

,Internalpatientid,Age at ed visit,Ed visit start date,Discharge date ed,Died during ed visit,CV diagnosis,code,cc Status
0,101689,64,2021-08-26 00:53:30,2021-08-26 04:24:27,0,1,I502,NCC
1,101689,64,2021-08-26 00:53:30,2021-08-26 04:24:27,0,1,N189,NCC
2,107210,71,2022-05-18 11:21:28,2022-05-18 18:30:21,0,0,A419,MCC
3,107210,71,2022-05-18 11:21:28,2022-05-18 18:30:21,0,0,I959,NCC
4,107866,61,2017-06-26 15:41:11,2017-06-26 23:33:50,0,0,R918,NCC
...,...,...,...,...,...,...,...,...
1099360,9883,90,2020-02-20 21:23:37,2020-02-20 22:56:30,0,0,W108,NCC
1099422,99235,87,2016-02-25 01:22:23,2016-02-25 06:33:45,0,0,R197,NCC
1099538,99902,66,2018-09-28 23:21:41,2018-09-28 23:56:55,0,0,M796,NCC
1099548,99934,84,2022-07-22 09:19:20,2022-07-22 09:42:57,0,0,M796,NCC


In [30]:
df = ed_visits_df.compute()
nan_percentage = calculate_nan_percentage(df)
nan_percentage

Internalpatientid       0.0
Age at ed visit         0.0
Ed visit start date     0.0
Discharge date ed       0.0
Died during ed visit    0.0
CV diagnosis            0.0
code                    0.0
cc Status               0.0
dtype: float64

In [31]:
ed_visits_df.dtypes

Internalpatientid                int32
Age at ed visit                   int8
Ed visit start date     datetime64[ns]
Discharge date ed       datetime64[ns]
Died during ed visit              int8
CV diagnosis                     int64
code                            object
cc Status                       object
dtype: object

In [32]:
# Save the Dask DataFrame as Parquet
ed_visits_df.to_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/ed_visits.parquet', engine='pyarrow')

# Immunizations

What do we want to do with these?

In [33]:
dataframes[5].columns

Index(['Internalpatientid', 'Age at immunization', 'Immunization date',
       'Immunization', 'Dose quantity', 'Dose unit', 'Administered elsewhere',
       'Cvx code', 'Series doses'],
      dtype='object')

In [53]:
df = dataframes[5].compute()
nan_percentage = calculate_nan_percentage(df)
nan_percentage

Internalpatientid          0.000000
Age at immunization        0.000000
Immunization date          0.000000
Immunization               0.000162
Dose quantity             99.396746
Dose unit                 99.396908
Administered elsewhere    85.789694
Cvx code                   0.991641
Series doses               0.000000
dtype: float64

Dose quantitity and Dose Unit are both primarily missing so I'll just drop those columns.

Administered elsewhere also has a lot of missing data. I might just drop it as well since that won't be too informative.

Cvx Code has a few NaNs. I'll just drop those.

In [54]:
df['Administered elsewhere'].value_counts()

Private Physician       9818
Private MD              5972
Walgreens #             5126
Non-VA Site             4972
Outside VA Result       3727
                        ... 
San ysidro health          1
Records                    1
CFH                        1
V.A. Clinic in Tyler       1
Eye Clinic                 1
Name: Administered elsewhere, Length: 31779, dtype: int64

Hmmm, consult with Jeff. I don't know how useful this is though. I could basically treat it as, 'if this value exists then the vaccine was administered elsewhere. Otherwise, it was administered at the VA'

In [55]:
df.dtypes

Internalpatientid                  int32
Age at immunization                 int8
Immunization date         datetime64[ns]
Immunization                      object
Dose quantity                    float64
Dose unit                         object
Administered elsewhere            object
Cvx code                         float64
Series doses                      object
dtype: object

In [56]:
df = df.drop(columns=['Dose quantity','Dose unit'])

In [57]:
df

,Internalpatientid,Age at immunization,Immunization date,Immunization,Administered elsewhere,Cvx code,Series doses
0,100000,64,2019-10-03 07:20:00,"PNEUMOCOCCAL, UNSPECIFIED FORMULATION",None,109.0,Not specified (no value)
1,100000,64,2019-10-03 07:20:00,PNEUMOCOCCAL POLYSACCHARIDE PPV23,None,33.0,Not specified (no value)
2,100001,84,2010-05-23 19:04:26,"INFLUENZA, UNSPECIFIED FORMULATION",None,88.0,Not specified (no value)
3,100008,71,2019-10-06 08:44:11,"INFLUENZA, INJECTABLE, QUADRIVALENT, PRESERVAT...",None,150.0,Not specified (no value)
4,100008,72,2020-10-05 03:49:18,"INFLUENZA, INJECTABLE, QUADRIVALENT, PRESERVAT...",None,150.0,Complete
...,...,...,...,...,...,...,...
615499,99988,77,2003-12-20 13:27:45,"INFLUENZA, UNSPECIFIED FORMULATION",None,88.0,Not specified (no value)
615500,99993,59,2008-01-24 05:19:38,"PNEUMOCOCCAL, UNSPECIFIED FORMULATION",None,109.0,Not specified (no value)
615501,99999,88,2004-11-27 04:28:15,"INFLUENZA, UNSPECIFIED FORMULATION",None,88.0,Not specified (no value)
615502,99999,90,2006-11-25 00:53:27,"INFLUENZA, UNSPECIFIED FORMULATION",None,88.0,Not specified (no value)


In [58]:
df['Cvx code'].describe()

count    1.829323e+06
mean     1.067990e+02
std      4.040394e+01
min      1.000000e+00
25%      8.800000e+01
50%      8.800000e+01
75%      1.330000e+02
max      5.000000e+02
Name: Cvx code, dtype: float64

Hmm, CVx code should probably be an int16

In [59]:
df = df.dropna(subset=['Cvx code'])

In [60]:
df['Cvx code'] = df['Cvx code'].astype('int16')

<ipython-input-60-ec4d638d2580>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cvx code'] = df['Cvx code'].astype('int16')


In [61]:
df['Series doses'].value_counts()

Not specified (no value)    1630582
1                             58575
Complete                      50766
2                             50009
3                             20240
Booster                       14433
4                              4091
Partial                         610
5                                13
6                                 4
Name: Series doses, dtype: int64

I'll change the 'Not specified' to NS for now. I don't know if we'll do anything with this.

In [ ]:
df['Series doses'] = df['Series doses'].replace('Not specified (no value)', 'NS')

In [63]:
df

,Internalpatientid,Age at immunization,Immunization date,Immunization,Administered elsewhere,Cvx code,Series doses
0,100000,64,2019-10-03 07:20:00,"PNEUMOCOCCAL, UNSPECIFIED FORMULATION",None,109,NS
1,100000,64,2019-10-03 07:20:00,PNEUMOCOCCAL POLYSACCHARIDE PPV23,None,33,NS
2,100001,84,2010-05-23 19:04:26,"INFLUENZA, UNSPECIFIED FORMULATION",None,88,NS
3,100008,71,2019-10-06 08:44:11,"INFLUENZA, INJECTABLE, QUADRIVALENT, PRESERVAT...",None,150,NS
4,100008,72,2020-10-05 03:49:18,"INFLUENZA, INJECTABLE, QUADRIVALENT, PRESERVAT...",None,150,Complete
...,...,...,...,...,...,...,...
615499,99988,77,2003-12-20 13:27:45,"INFLUENZA, UNSPECIFIED FORMULATION",None,88,NS
615500,99993,59,2008-01-24 05:19:38,"PNEUMOCOCCAL, UNSPECIFIED FORMULATION",None,109,NS
615501,99999,88,2004-11-27 04:28:15,"INFLUENZA, UNSPECIFIED FORMULATION",None,88,NS
615502,99999,90,2006-11-25 00:53:27,"INFLUENZA, UNSPECIFIED FORMULATION",None,88,NS


In [49]:
df.dtypes

Internalpatientid                  int32
Age at immunization                 int8
Immunization date         datetime64[ns]
Immunization                      object
Administered elsewhere            object
Cvx code                           int16
Series doses                      object
dtype: object

For Series doses, I'll just convert these to categorical variables and call it good. We can simply remove those columns if not informative later.

In [50]:
#df['Series doses'] = 'Immu_' + df['Series doses'].astype(str)

<ipython-input-50-84ba50106d6d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Series doses'] = 'Immu_' + df['Series doses'].astype(str)


In [64]:
dummy_df  = pd.get_dummies(df['Series doses'], prefix='Series doses')
df = df.merge(dummy_df, left_index=True, right_index=True)
df = df.drop(columns=['Series doses'])
df

,Internalpatientid,Age at immunization,Immunization date,Immunization,Administered elsewhere,Cvx code,Series doses_1,Series doses_2,Series doses_3,Series doses_4,Series doses_5,Series doses_6,Series doses_Booster,Series doses_Complete,Series doses_NS,Series doses_Partial
0,100000,64,2019-10-03 07:20:00,"PNEUMOCOCCAL, UNSPECIFIED FORMULATION",None,109,0,0,0,0,0,0,0,0,1,0
0,100000,64,2019-10-03 07:20:00,"PNEUMOCOCCAL, UNSPECIFIED FORMULATION",None,109,0,0,0,0,0,0,0,0,1,0
0,100000,64,2019-10-03 07:20:00,"PNEUMOCOCCAL, UNSPECIFIED FORMULATION",None,109,0,0,0,0,0,0,0,0,1,0
0,162877,98,2013-10-17 00:23:48,"FLU,3 YRS (HISTORICAL)",None,88,0,0,0,0,0,0,0,0,1,0
0,162877,98,2013-10-17 00:23:48,"FLU,3 YRS (HISTORICAL)",None,88,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616475,162868,92,2020-11-29 05:20:37,ZOSTER RECOMBINANT,None,187,1,0,0,0,0,0,0,0,0,0
616476,162869,78,2007-03-16 22:39:20,ZZZINFLUENZA (HISTORICAL),None,88,0,0,0,0,0,0,0,0,1,0
616477,162875,60,2010-11-27 21:25:26,TD(ADULT) UNSPECIFIED FORMULATION,None,139,0,0,0,0,0,0,0,0,1,0
616478,162877,91,2007-01-24 05:40:58,TD(ADULT) UNSPECIFIED FORMULATION,None,139,0,0,0,0,0,0,0,0,1,0


In [65]:
# Save the Dask DataFrame as Parquet
df.to_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/immunizations.parquet', engine='pyarrow')

I'm not sure how we want to harmonize the Immunization since there's a bunch of similarly named vaccines there.

# Cleaning inpatient admissions DF

For this one I'll check for NaNs and map the ICD codes. Then we'll see.

In [66]:
dataframes[6].columns

Index(['Internalpatientid', 'Age at admission', 'Admission date',
       'Discharge date', 'Admitting unit service', 'Discharging unit service',
       'Admitting specialty', 'Discharging specialty',
       'First listed discharge diagnosis icd10 subcategory',
       'Second listed discharge diagnosis icd10 subcategory',
       'Discharge disposition', 'Died during admission',
       'Outpatientreferralflag', 'Serviceconnectedflag', 'Agentorangeflag',
       'CV diagnosis'],
      dtype='object')

In [67]:
dataframes[6].dtypes

Internalpatientid                                               int32
Age at admission                                                 int8
Admission date                                         datetime64[ns]
Discharge date                                         datetime64[ns]
Admitting unit service                                         object
Discharging unit service                                       object
Admitting specialty                                            object
Discharging specialty                                          object
First listed discharge diagnosis icd10 subcategory             object
Second listed discharge diagnosis icd10 subcategory            object
Discharge disposition                                          object
Died during admission                                          object
Outpatientreferralflag                                         object
Serviceconnectedflag                                           object
Agentorangeflag     

I need to convert CV diagnosis to an int8.

Died during admission, outpatientreferralflag, serviceconnectedflag, agentorangeflag are categorical and need to be converted as well.

In [ ]:
inpatient_df = dataframes[6]
inpatient_df = process_diagnoses_dataframe(inpatient_df, icd_codes)

I'm dropping the ServiceConnectedFlag.
For w

In [70]:
inpatient_df = inpatient_df.compute()
inpatient_df

,Internalpatientid,Age at admission,Admission date,Discharge date,Admitting unit service,Discharging unit service,Admitting specialty,Discharging specialty,Discharge disposition,Died during admission,Outpatientreferralflag,Serviceconnectedflag,Agentorangeflag,CV diagnosis,diagnosis,code,billable,cc Status
0,10,66,2015-11-28 17:41:09,2015-11-29 01:43:14,NON-COUNT,NON-COUNT,DRUG DEPENDENCE TRMT UNIT,MEDICAL OBSERVATION,Regular,No,No,None,Yes,0,pneumonia unspecified organism,J189,1.0,MCC
1,10,66,2015-11-28 17:41:09,2015-11-29 01:43:14,NON-COUNT,NON-COUNT,DRUG DEPENDENCE TRMT UNIT,MEDICAL OBSERVATION,Regular,No,No,None,Yes,0,hypokalemia,E876,1.0,NCC
2,100001,84,2009-10-01 21:19:50,2009-10-04 16:51:33,MEDICINE,MEDICINE,PSYCHIATRIC MENTALLY INFIRM,GENERAL(ACUTE MEDICINE),Regular,No,Yes,No,No,0,pneumonia unspecified organism,J189,1.0,MCC
3,100001,84,2009-10-01 21:19:50,2009-10-04 16:51:33,MEDICINE,MEDICINE,PSYCHIATRIC MENTALLY INFIRM,GENERAL(ACUTE MEDICINE),Regular,No,Yes,No,No,0,essential (primary) hypertension,I10,1.0,NCC
4,100001,85,2010-11-10 04:32:39,2010-11-19 08:49:45,SURGERY,SURGERY,SUBSTANCE ABUSE RES TRMT PROG,ORTHOPEDIC,Regular,No,No,None,No,0,osteoarthritis unspecified site,M199,0.0,NCC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085813,99975,89,2018-01-05 00:59:05,2018-01-06 23:18:39,NON-COUNT,NON-COUNT,PODIATRY,MEDICAL OBSERVATION,Regular,No,No,None,No,0,essential (primary) hypertension,I10,1.0,NCC
1085814,99986,73,2019-06-26 19:47:10,2019-06-29 21:18:00,MEDICINE,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),Regular,No,Yes,None,No,0,cellulitis and acute lymphangitis of other par...,L031,0.0,NCC
1085815,99986,73,2019-06-26 19:47:10,2019-06-29 21:18:00,MEDICINE,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),Regular,No,Yes,None,No,0,other chronic osteomyelitis,M866,0.0,NCC
1085816,99994,83,2016-08-13 20:09:52,2016-08-15 16:43:07,SURGERY,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),Regular,No,Yes,None,No,0,urinary tract infection site not specified,N390,1.0,CC


In [78]:
nan_percentage = calculate_nan_percentage(inpatient_df)
nan_percentage

Internalpatientid            0.000000
Age at admission             0.000000
Admission date               0.000000
Discharge date               0.091083
Admitting unit service       0.000000
Discharging unit service     0.000000
Admitting specialty          0.000000
Discharging specialty        0.000000
Discharge disposition        0.000000
Died during admission        0.000000
Outpatientreferralflag       5.019257
Serviceconnectedflag        93.196189
Agentorangeflag             20.259657
CV diagnosis                 0.000000
diagnosis                    0.000000
code                         2.707084
billable                     2.707084
cc Status                    2.707084
dtype: float64

In [81]:
inpatient_df['CV diagnosis'] = inpatient_df['CV diagnosis'].astype('int8')

In [ ]:
inpatient_df = inpatient_df.drop(columns=['Serviceconnectedflag','billable'])
inpatient_df = inpatient_df.dropna(subset=['code','Outpatientreferralflag'])

In [83]:
nan_percentage = calculate_nan_percentage(inpatient_df)
nan_percentage

Internalpatientid            0.000000
Age at admission             0.000000
Admission date               0.000000
Discharge date               0.004084
Admitting unit service       0.000000
Discharging unit service     0.000000
Admitting specialty          0.000000
Discharging specialty        0.000000
Discharge disposition        0.000000
Died during admission        0.000000
Outpatientreferralflag       0.000000
Agentorangeflag             19.813498
CV diagnosis                 0.000000
diagnosis                    0.000000
code                         0.000000
cc Status                    0.000000
dtype: float64

In [71]:
inpatient_df['Discharge disposition'].value_counts()

Regular                     1002208
Death without autopsy         30517
Transfer                      24944
Irregular                     20443
NBC or while ASIH              4737
Death with autopsy             1973
Not specified (no value)        996
Name: Discharge disposition, dtype: int64

Hmmm, convert to categorical and then get dummies? Maybe drop the Not specified column?

In [72]:
inpatient_df['Died during admission'].value_counts()

No     1052022
Yes      33796
Name: Died during admission, dtype: int64

Map these to a simple binary

In [73]:
inpatient_df['Outpatientreferralflag'].value_counts()

Yes    815537
No     215781
Name: Outpatientreferralflag, dtype: int64

Map these to a simple binary

In [74]:
inpatient_df['Serviceconnectedflag'].value_counts()

No     66755
Yes     7122
Name: Serviceconnectedflag, dtype: int64

Map these to a simple binary

In [75]:
inpatient_df['Agentorangeflag'].value_counts()

No     776045
Yes     89790
Name: Agentorangeflag, dtype: int64

Map to a simple binary and inpute missing values with the mode

In [85]:
mode = inpatient_df['Agentorangeflag'].mode().iloc[0]
inpatient_df['Agentorangeflag'] = inpatient_df['Agentorangeflag'].fillna(mode)

In [86]:
nan_percentage = calculate_nan_percentage(inpatient_df)
nan_percentage

Internalpatientid           0.000000
Age at admission            0.000000
Admission date              0.000000
Discharge date              0.004084
Admitting unit service      0.000000
Discharging unit service    0.000000
Admitting specialty         0.000000
Discharging specialty       0.000000
Discharge disposition       0.000000
Died during admission       0.000000
Outpatientreferralflag      0.000000
Agentorangeflag             0.000000
CV diagnosis                0.000000
diagnosis                   0.000000
code                        0.000000
cc Status                   0.000000
dtype: float64

I'll just drop the rows in Discarge date with a NaN. Not worth worrying about right now.

In [87]:
inpatient_df = inpatient_df.dropna(subset=['Discharge date'])

In [92]:
for col in ['Died during admission', 'Outpatientreferralflag', 'Agentorangeflag']:
  inpatient_df = inpatient_df[~((inpatient_df[col].isna()) | (inpatient_df[col] == np.inf) | (inpatient_df[col] == -np.inf))]

In [93]:
nan_percentage = calculate_nan_percentage(inpatient_df)
nan_percentage

Internalpatientid           0.0
Age at admission            0.0
Admission date              0.0
Discharge date              0.0
Admitting unit service      0.0
Discharging unit service    0.0
Admitting specialty         0.0
Discharging specialty       0.0
Discharge disposition       0.0
Died during admission       0.0
Outpatientreferralflag      0.0
Agentorangeflag             0.0
CV diagnosis                0.0
diagnosis                   0.0
code                        0.0
cc Status                   0.0
dtype: float64

No more missing values.

In [97]:
inpatient_df

,Internalpatientid,Age at admission,Admission date,Discharge date,Admitting unit service,Discharging unit service,Admitting specialty,Discharging specialty,Discharge disposition,Died during admission,Outpatientreferralflag,Agentorangeflag,CV diagnosis,diagnosis,code,cc Status
0,10,66,2015-11-28 17:41:09,2015-11-29 01:43:14,NON-COUNT,NON-COUNT,DRUG DEPENDENCE TRMT UNIT,MEDICAL OBSERVATION,Regular,0,No,Yes,0,pneumonia unspecified organism,J189,MCC
1,10,66,2015-11-28 17:41:09,2015-11-29 01:43:14,NON-COUNT,NON-COUNT,DRUG DEPENDENCE TRMT UNIT,MEDICAL OBSERVATION,Regular,0,No,Yes,0,hypokalemia,E876,NCC
2,100001,84,2009-10-01 21:19:50,2009-10-04 16:51:33,MEDICINE,MEDICINE,PSYCHIATRIC MENTALLY INFIRM,GENERAL(ACUTE MEDICINE),Regular,0,Yes,No,0,pneumonia unspecified organism,J189,MCC
3,100001,84,2009-10-01 21:19:50,2009-10-04 16:51:33,MEDICINE,MEDICINE,PSYCHIATRIC MENTALLY INFIRM,GENERAL(ACUTE MEDICINE),Regular,0,Yes,No,0,essential (primary) hypertension,I10,NCC
4,100001,85,2010-11-10 04:32:39,2010-11-19 08:49:45,SURGERY,SURGERY,SUBSTANCE ABUSE RES TRMT PROG,ORTHOPEDIC,Regular,0,No,No,0,osteoarthritis unspecified site,M199,NCC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085813,99975,89,2018-01-05 00:59:05,2018-01-06 23:18:39,NON-COUNT,NON-COUNT,PODIATRY,MEDICAL OBSERVATION,Regular,0,No,No,0,essential (primary) hypertension,I10,NCC
1085814,99986,73,2019-06-26 19:47:10,2019-06-29 21:18:00,MEDICINE,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),Regular,0,Yes,No,0,cellulitis and acute lymphangitis of other par...,L031,NCC
1085815,99986,73,2019-06-26 19:47:10,2019-06-29 21:18:00,MEDICINE,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),Regular,0,Yes,No,0,other chronic osteomyelitis,M866,NCC
1085816,99994,83,2016-08-13 20:09:52,2016-08-15 16:43:07,SURGERY,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),Regular,0,Yes,No,0,urinary tract infection site not specified,N390,CC


In [98]:
mapping = {'No': 0, 'Yes': 1}

for col in ['Outpatientreferralflag', 'Agentorangeflag']:
    print(col)
    inpatient_df[col] = inpatient_df[col].map(mapping).astype('int8')

Outpatientreferralflag
Agentorangeflag


In [99]:
inpatient_df

,Internalpatientid,Age at admission,Admission date,Discharge date,Admitting unit service,Discharging unit service,Admitting specialty,Discharging specialty,Discharge disposition,Died during admission,Outpatientreferralflag,Agentorangeflag,CV diagnosis,diagnosis,code,cc Status
0,10,66,2015-11-28 17:41:09,2015-11-29 01:43:14,NON-COUNT,NON-COUNT,DRUG DEPENDENCE TRMT UNIT,MEDICAL OBSERVATION,Regular,0,0,1,0,pneumonia unspecified organism,J189,MCC
1,10,66,2015-11-28 17:41:09,2015-11-29 01:43:14,NON-COUNT,NON-COUNT,DRUG DEPENDENCE TRMT UNIT,MEDICAL OBSERVATION,Regular,0,0,1,0,hypokalemia,E876,NCC
2,100001,84,2009-10-01 21:19:50,2009-10-04 16:51:33,MEDICINE,MEDICINE,PSYCHIATRIC MENTALLY INFIRM,GENERAL(ACUTE MEDICINE),Regular,0,1,0,0,pneumonia unspecified organism,J189,MCC
3,100001,84,2009-10-01 21:19:50,2009-10-04 16:51:33,MEDICINE,MEDICINE,PSYCHIATRIC MENTALLY INFIRM,GENERAL(ACUTE MEDICINE),Regular,0,1,0,0,essential (primary) hypertension,I10,NCC
4,100001,85,2010-11-10 04:32:39,2010-11-19 08:49:45,SURGERY,SURGERY,SUBSTANCE ABUSE RES TRMT PROG,ORTHOPEDIC,Regular,0,0,0,0,osteoarthritis unspecified site,M199,NCC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085813,99975,89,2018-01-05 00:59:05,2018-01-06 23:18:39,NON-COUNT,NON-COUNT,PODIATRY,MEDICAL OBSERVATION,Regular,0,0,0,0,essential (primary) hypertension,I10,NCC
1085814,99986,73,2019-06-26 19:47:10,2019-06-29 21:18:00,MEDICINE,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),Regular,0,1,0,0,cellulitis and acute lymphangitis of other par...,L031,NCC
1085815,99986,73,2019-06-26 19:47:10,2019-06-29 21:18:00,MEDICINE,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),Regular,0,1,0,0,other chronic osteomyelitis,M866,NCC
1085816,99994,83,2016-08-13 20:09:52,2016-08-15 16:43:07,SURGERY,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),Regular,0,1,0,0,urinary tract infection site not specified,N390,CC


In [100]:
inpatient_df['cc Status'].value_counts()

NCC    749832
CC     175606
MCC     78478
Name: cc Status, dtype: int64

I'll set cc Status to categorical and make dummies

In [101]:
dummy_df  = pd.get_dummies(inpatient_df['cc Status'], prefix='cc Status')
inpatient_df = inpatient_df.merge(dummy_df, left_index=True, right_index=True)
inpatient_df = inpatient_df.drop(columns=['cc Status'])
inpatient_df

,Internalpatientid,Age at admission,Admission date,Discharge date,Admitting unit service,Discharging unit service,Admitting specialty,Discharging specialty,Discharge disposition,Died during admission,Outpatientreferralflag,Agentorangeflag,CV diagnosis,diagnosis,code,cc Status_CC,cc Status_MCC,cc Status_NCC
0,10,66,2015-11-28 17:41:09,2015-11-29 01:43:14,NON-COUNT,NON-COUNT,DRUG DEPENDENCE TRMT UNIT,MEDICAL OBSERVATION,Regular,0,0,1,0,pneumonia unspecified organism,J189,0,1,0
1,10,66,2015-11-28 17:41:09,2015-11-29 01:43:14,NON-COUNT,NON-COUNT,DRUG DEPENDENCE TRMT UNIT,MEDICAL OBSERVATION,Regular,0,0,1,0,hypokalemia,E876,0,0,1
2,100001,84,2009-10-01 21:19:50,2009-10-04 16:51:33,MEDICINE,MEDICINE,PSYCHIATRIC MENTALLY INFIRM,GENERAL(ACUTE MEDICINE),Regular,0,1,0,0,pneumonia unspecified organism,J189,0,1,0
3,100001,84,2009-10-01 21:19:50,2009-10-04 16:51:33,MEDICINE,MEDICINE,PSYCHIATRIC MENTALLY INFIRM,GENERAL(ACUTE MEDICINE),Regular,0,1,0,0,essential (primary) hypertension,I10,0,0,1
4,100001,85,2010-11-10 04:32:39,2010-11-19 08:49:45,SURGERY,SURGERY,SUBSTANCE ABUSE RES TRMT PROG,ORTHOPEDIC,Regular,0,0,0,0,osteoarthritis unspecified site,M199,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085813,99975,89,2018-01-05 00:59:05,2018-01-06 23:18:39,NON-COUNT,NON-COUNT,PODIATRY,MEDICAL OBSERVATION,Regular,0,0,0,0,essential (primary) hypertension,I10,0,0,1
1085814,99986,73,2019-06-26 19:47:10,2019-06-29 21:18:00,MEDICINE,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),Regular,0,1,0,0,cellulitis and acute lymphangitis of other par...,L031,0,0,1
1085815,99986,73,2019-06-26 19:47:10,2019-06-29 21:18:00,MEDICINE,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),Regular,0,1,0,0,other chronic osteomyelitis,M866,0,0,1
1085816,99994,83,2016-08-13 20:09:52,2016-08-15 16:43:07,SURGERY,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),Regular,0,1,0,0,urinary tract infection site not specified,N390,1,0,0


In [102]:
inpatient_df.dtypes

Internalpatientid                    int32
Age at admission                      int8
Admission date              datetime64[ns]
Discharge date              datetime64[ns]
Admitting unit service              object
Discharging unit service            object
Admitting specialty                 object
Discharging specialty               object
Discharge disposition               object
Died during admission                 int8
Outpatientreferralflag                int8
Agentorangeflag                       int8
CV diagnosis                          int8
diagnosis                           object
code                                object
cc Status_CC                         uint8
cc Status_MCC                        uint8
cc Status_NCC                        uint8
dtype: object

In [103]:
inpatient_df['Admitting unit service'].value_counts()

MEDICINE              616959
SURGERY               134153
NON-COUNT             126722
PSYCHIATRY             47023
NHCU                   44448
DOMICILIARY            13490
SPINAL CORD INJURY      6127
INTERMEDIATE MED        4916
NEUROLOGY               4916
REHAB MEDICINE          2410
BLIND REHAB             2059
(Censored)               655
Not specified             38
Name: Admitting unit service, dtype: int64

In [105]:
inpatient_df = inpatient_df[~((inpatient_df['Admitting unit service'] == '(Censored)') | (inpatient_df['Admitting unit service'] == 'Not specified'))]

Need to remove censored or not specified Admit unit

In [104]:
inpatient_df['Discharging unit service'].value_counts()

MEDICINE                    606009
SURGERY                     133818
NON-COUNT                   125594
PSYCHIATRY                   49271
NHCU                         40430
INTERMEDIATE MED             13502
DOMICILIARY                  13084
SPINAL CORD INJURY            6966
REHAB MEDICINE                5754
NEUROLOGY                     5078
BLIND REHAB                   2020
Not specified (no value)      1789
(Censored)                     561
Not specified                   40
Name: Discharging unit service, dtype: int64

Need to remove censored or not specified or Not specified (no value) from Discharge unit

In [107]:
inpatient_df = inpatient_df[~((inpatient_df['Discharging unit service'] == '(Censored)') | (inpatient_df['Discharging unit service'] == 'Not specified') | (inpatient_df['Discharging unit service'] == 'Not specified (no value)'))]

In [109]:
inpatient_df['Admitting specialty'].value_counts()

BLIND REHAB OBSERVATION          21232
PULMONARY, TUBERCULOSIS          20876
INTERMEDIATE MEDICINE            19967
SPINAL CORD INJURY               19416
NH SHORT-STAY CONTINUING CARE    18848
                                 ...  
zSUBST ABUSE STAR I, II & III       86
Not specified (no value)            50
HIGH INTENSITY GEN INPT             35
PSYCHIATRY                          14
ZZSUBST ABUSE STAR I,II,II           5
Name: Admitting specialty, Length: 124, dtype: int64

In [110]:
inpatient_df['Discharging specialty'].value_counts()

GENERAL(ACUTE MEDICINE)           423273
MEDICAL OBSERVATION               105731
Not specified (no value)           97021
TELEMETRY                          37446
HIGH INTENSITY GEN PSYCH INPAT     35602
                                   ...  
DOMICILIARY GENERAL                    6
LONG STAY GRECC-NHCU                   6
REHAB MEDICINE OBSERVATION             4
DERMATOLOGY                            3
ALLERGY                                2
Name: Discharging specialty, Length: 100, dtype: int64

In [111]:
inpatient_df['Discharge disposition'].value_counts()

Regular                     972278
Transfer                     24344
NBC or while ASIH             4593
Death without autopsy           84
Irregular                       62
Death with autopsy               2
Not specified (no value)         2
Name: Discharge disposition, dtype: int64

Drop not specified

In [112]:
inpatient_df = inpatient_df[~(inpatient_df['Discharge disposition'] == 'Not specified (no value)')]

Now I'll expand the Discharge disposition into dummy variables

In [116]:
dummy_df  = pd.get_dummies(inpatient_df['Discharge disposition'], prefix='Discharge disposition')
inpatient_df = inpatient_df.merge(dummy_df, left_index=True, right_index=True)
inpatient_df = inpatient_df.drop(columns=['Discharge disposition'])
inpatient_df

,Internalpatientid,Age at admission,Admission date,Discharge date,Admitting unit service,Discharging unit service,Admitting specialty,Discharging specialty,Died during admission,Outpatientreferralflag,...,code,cc Status_CC,cc Status_MCC,cc Status_NCC,Discharge disposition_Death with autopsy,Discharge disposition_Death without autopsy,Discharge disposition_Irregular,Discharge disposition_NBC or while ASIH,Discharge disposition_Regular,Discharge disposition_Transfer
0,10,66,2015-11-28 17:41:09,2015-11-29 01:43:14,NON-COUNT,NON-COUNT,DRUG DEPENDENCE TRMT UNIT,MEDICAL OBSERVATION,0,0,...,J189,0,1,0,0,0,0,0,1,0
1,10,66,2015-11-28 17:41:09,2015-11-29 01:43:14,NON-COUNT,NON-COUNT,DRUG DEPENDENCE TRMT UNIT,MEDICAL OBSERVATION,0,0,...,E876,0,0,1,0,0,0,0,1,0
2,100001,84,2009-10-01 21:19:50,2009-10-04 16:51:33,MEDICINE,MEDICINE,PSYCHIATRIC MENTALLY INFIRM,GENERAL(ACUTE MEDICINE),0,1,...,J189,0,1,0,0,0,0,0,1,0
3,100001,84,2009-10-01 21:19:50,2009-10-04 16:51:33,MEDICINE,MEDICINE,PSYCHIATRIC MENTALLY INFIRM,GENERAL(ACUTE MEDICINE),0,1,...,I10,0,0,1,0,0,0,0,1,0
4,100001,85,2010-11-10 04:32:39,2010-11-19 08:49:45,SURGERY,SURGERY,SUBSTANCE ABUSE RES TRMT PROG,ORTHOPEDIC,0,0,...,M199,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085813,99975,89,2018-01-05 00:59:05,2018-01-06 23:18:39,NON-COUNT,NON-COUNT,PODIATRY,MEDICAL OBSERVATION,0,0,...,I10,0,0,1,0,0,0,0,1,0
1085814,99986,73,2019-06-26 19:47:10,2019-06-29 21:18:00,MEDICINE,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),0,1,...,L031,0,0,1,0,0,0,0,1,0
1085815,99986,73,2019-06-26 19:47:10,2019-06-29 21:18:00,MEDICINE,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),0,1,...,M866,0,0,1,0,0,0,0,1,0
1085816,99994,83,2016-08-13 20:09:52,2016-08-15 16:43:07,SURGERY,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),0,1,...,N390,1,0,0,0,0,0,0,1,0


In [115]:
(1085818 - 1001363)/1085818  * 100

7.778006995647521

In [117]:
inpatient_df.dtypes

Internalpatientid                                       int32
Age at admission                                         int8
Admission date                                 datetime64[ns]
Discharge date                                 datetime64[ns]
Admitting unit service                                 object
Discharging unit service                               object
Admitting specialty                                    object
Discharging specialty                                  object
Died during admission                                    int8
Outpatientreferralflag                                   int8
Agentorangeflag                                          int8
CV diagnosis                                             int8
diagnosis                                              object
code                                                   object
cc Status_CC                                            uint8
cc Status_MCC                                           uint8
cc Statu

In [118]:
# Save the Dask DataFrame as Parquet
inpatient_df.to_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/inpatient_admissions.parquet', engine='pyarrow')

# Cleaning Inpatient Location



In [119]:
dataframes[7].columns

Index(['Internalpatientid', 'Age at location', 'Location start date',
       'Location end date', 'Service', 'Died at location'],
      dtype='object')

In [121]:
inpatient_location_df = dataframes[7].compute()

In [122]:
inpatient_location_df

,Internalpatientid,Age at location,Location start date,Location end date,Service,Died at location
0,100005,66,2002-09-09 01:24:31,2002-09-12 01:00:40,MEDICINE,No
1,100008,70,2019-06-04 05:46:59,2019-06-05 10:07:31,SURGERY,No
2,100015,55,2001-04-01 01:32:23,2001-04-02 20:04:22,MEDICINE,No
3,100015,56,2001-07-29 20:19:52,2001-07-30 22:42:13,MEDICINE,No
4,100020,61,2022-04-05 21:55:19,NaT,SURGERY,No
...,...,...,...,...,...,...
850032,99974,53,2003-08-25 17:35:48,2003-08-31 15:18:56,MEDICINE,No
850033,99981,67,2009-08-21 05:12:20,2009-08-24 07:10:02,SURGERY,No
850034,99983,44,2006-04-22 23:06:15,2006-04-29 13:25:45,DOMICILIARY,No
850035,9999,61,2013-09-14 10:54:27,2013-09-15 06:16:09,SURGERY,No


In [123]:
nan_percentage = calculate_nan_percentage(inpatient_location_df)
nan_percentage

Internalpatientid      0.000000
Age at location        0.000000
Location start date    0.000000
Location end date      2.260372
Service                0.000000
Died at location       0.000000
dtype: float64

I'll drop location end date NaNs

In [127]:
inpatient_location_df = inpatient_location_df.dropna(subset=['Location end date'])

Check dtypes. Died at location needs to be converted into categorical. Check Service for any unspecified/censored values

In [124]:
inpatient_location_df.dtypes

Internalpatientid               int32
Age at location                  int8
Location start date    datetime64[ns]
Location end date      datetime64[ns]
Service                        object
Died at location               object
dtype: object

In [125]:
inpatient_location_df['Service'].value_counts()

MEDICINE                    477243
SURGERY                     132110
NON-COUNT                    74145
NHCU                         72722
PSYCHIATRY                   34505
DOMICILIARY                  33405
INTERMEDIATE MED              9230
SPINAL CORD INJURY            5247
REHAB MEDICINE                3863
NEUROLOGY                     3577
BLIND REHAB                   3450
(Censored)                     318
Not specified                  180
Not specified (no value)        42
Name: Service, dtype: int64

Need to dtop censored, and not specified values from Service column

In [126]:
inpatient_location_df = inpatient_location_df[~((inpatient_location_df['Service'] == '(Censored)') | (inpatient_location_df['Service'] == 'Not specified') | (inpatient_location_df['Service'] == 'Not specified (no value)'))]

Now I'll convert Died at location to categorical, and change it to int8

In [128]:
inpatient_location_df['Died at location'].value_counts()

No     818131
Yes     12222
Name: Died at location, dtype: int64

In [129]:
mapping = {'No': 0, 'Yes': 1}
inpatient_location_df['Died at location'] = inpatient_location_df['Died at location'].map(mapping).astype('int8')

In [130]:
inpatient_location_df

,Internalpatientid,Age at location,Location start date,Location end date,Service,Died at location
0,100005,66,2002-09-09 01:24:31,2002-09-12 01:00:40,MEDICINE,0
1,100008,70,2019-06-04 05:46:59,2019-06-05 10:07:31,SURGERY,0
2,100015,55,2001-04-01 01:32:23,2001-04-02 20:04:22,MEDICINE,0
3,100015,56,2001-07-29 20:19:52,2001-07-30 22:42:13,MEDICINE,0
5,100026,61,2017-04-22 12:58:18,2017-04-25 03:43:53,MEDICINE,0
...,...,...,...,...,...,...
850032,99974,53,2003-08-25 17:35:48,2003-08-31 15:18:56,MEDICINE,0
850033,99981,67,2009-08-21 05:12:20,2009-08-24 07:10:02,SURGERY,0
850034,99983,44,2006-04-22 23:06:15,2006-04-29 13:25:45,DOMICILIARY,0
850035,9999,61,2013-09-14 10:54:27,2013-09-15 06:16:09,SURGERY,0


In [132]:
inpatient_location_df['Service'].value_counts()

MEDICINE              464201
SURGERY               129180
NON-COUNT              73047
NHCU                   71792
PSYCHIATRY             33988
DOMICILIARY            33233
INTERMEDIATE MED        9132
SPINAL CORD INJURY      5095
REHAB MEDICINE          3747
NEUROLOGY               3516
BLIND REHAB             3422
Name: Service, dtype: int64

In [133]:
inpatient_location_df.dtypes

Internalpatientid               int32
Age at location                  int8
Location start date    datetime64[ns]
Location end date      datetime64[ns]
Service                        object
Died at location                 int8
dtype: object

That's enough with this for now.

In [134]:
# Save the Dask DataFrame as Parquet
inpatient_location_df.to_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/inpatient_location.parquet', engine='pyarrow')

# Cleaning Inpatient Specialty

In [135]:
dataframes[8].columns

Index(['Internalpatientid', 'Age at specialty', 'Specialty start date',
       'Specialty end date', 'Specialty'],
      dtype='object')

In [136]:
inpatient_specialty_df = dataframes[8].compute()
inpatient_specialty_df

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty
0,1,79,2022-12-31 05:41:51,2023-01-01 23:06:29,DERMATOLOGY
1,100001,87,2012-07-21 23:00:00,2012-07-22 19:18:47,PM&R TRANSITIONAL REHAB
2,100015,57,2002-07-16 06:40:13,2002-07-16 06:40:13,INTERMEDIATE MEDICINE
3,100015,58,2004-02-21 11:01:35,2004-02-22 14:56:40,DOMICILIARY PTSD
4,100019,81,1999-10-24 10:44:24,1999-10-31 10:20:31,PLASTIC SURGERY
...,...,...,...,...,...
725466,99993,58,2007-02-18 00:35:16,2007-02-18 04:55:16,METABOLIC
725467,99994,83,2016-10-18 10:35:16,2016-10-21 11:02:57,NH SHORT STAY DEMENTIA CARE
725468,99994,86,2019-08-30 08:38:03,2019-09-05 09:28:48,"PULMONARY, NON-TB"
725469,99996,56,2011-08-17 00:10:52,2011-08-17 00:10:52,GENERAL(ACUTE MEDICINE)


In [137]:
nan_percentage = calculate_nan_percentage(inpatient_specialty_df)
nan_percentage

Internalpatientid       0.000000
Age at specialty        0.000000
Specialty start date    0.000000
Specialty end date      0.030078
Specialty               0.000000
dtype: float64

In [138]:
inpatient_specialty_df = inpatient_specialty_df.dropna(subset=['Specialty end date'])

In [141]:
inpatient_specialty_df.dtypes

Internalpatientid                int32
Age at specialty                  int8
Specialty start date    datetime64[ns]
Specialty end date      datetime64[ns]
Specialty                       object
dtype: object

In [139]:
inpatient_specialty_df['Specialty'].value_counts()

GENERAL(ACUTE MEDICINE)         249188
MEDICAL OBSERVATION              62378
TELEMETRY                        32400
GENERAL SURGERY                  30765
INTERMEDIATE MEDICINE            30746
                                 ...  
zGENERAL(ACUTE MEDICINE             15
ZZSUBST ABUSE STAR I,II,II           6
DOMICILIARY SUBSTANCE USE DO         4
DOMICILIARY GENERAL                  4
ZZ POLYTRAUMA REHAB UNIT             1
Name: Specialty, Length: 128, dtype: int64

In [140]:
inpatient_specialty_df = inpatient_specialty_df[~((inpatient_specialty_df['Specialty'] == '(Censored)') | (inpatient_specialty_df['Specialty'] == 'Not specified') | (inpatient_specialty_df['Specialty'] == 'Not specified (no value)'))]

In [142]:
inpatient_specialty_df

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty
0,1,79,2022-12-31 05:41:51,2023-01-01 23:06:29,DERMATOLOGY
1,100001,87,2012-07-21 23:00:00,2012-07-22 19:18:47,PM&R TRANSITIONAL REHAB
2,100015,57,2002-07-16 06:40:13,2002-07-16 06:40:13,INTERMEDIATE MEDICINE
3,100015,58,2004-02-21 11:01:35,2004-02-22 14:56:40,DOMICILIARY PTSD
4,100019,81,1999-10-24 10:44:24,1999-10-31 10:20:31,PLASTIC SURGERY
...,...,...,...,...,...
725466,99993,58,2007-02-18 00:35:16,2007-02-18 04:55:16,METABOLIC
725467,99994,83,2016-10-18 10:35:16,2016-10-21 11:02:57,NH SHORT STAY DEMENTIA CARE
725468,99994,86,2019-08-30 08:38:03,2019-09-05 09:28:48,"PULMONARY, NON-TB"
725469,99996,56,2011-08-17 00:10:52,2011-08-17 00:10:52,GENERAL(ACUTE MEDICINE)


That's it for now.

In [145]:
# Save the Dask DataFrame as Parquet
inpatient_specialty_df.to_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/inpatient_specialty.parquet', engine='pyarrow')

# Cleaning for Measurements DF

In [146]:
dataframes[9].columns

Index(['Internalpatientid', 'Age at measurement', 'Measurement date',
       'Measurement', 'Result numeric', 'Result textual'],
      dtype='object')

In [6]:
measurements_df = dataframes[9].compute()

I'll just drop any missing values from age, measurement date, measurement and result numeric.

In [7]:
columns_list = ["Age at measurement", "Measurement date", "Measurement", "Result numeric"]
measurements_df = measurements_df.dropna(subset = columns_list)

In [149]:
measurements_df['Result textual'].value_counts()

Series([], Name: Result textual, dtype: int64)

Hmmm, Result textual seems to be completely empty... Let me double check

In [152]:
nan_percentage = calculate_nan_percentage(measurements_df)
nan_percentage

Internalpatientid       0.0
Age at measurement      0.0
Measurement date        0.0
Measurement             0.0
Result numeric          0.0
Result textual        100.0
dtype: float64

Yeah, that column is useless. I'm removing it

In [8]:
measurements_df = measurements_df.drop(columns=['Result textual'])

I'll convert the Measurement column to categorical and make dummies and then move on for now.

In [9]:
measurements_df = dd.from_pandas(measurements_df, npartitions=100)
measurements_df['Measurement'] = measurements_df['Measurement'].astype('category')
measurements_df['Measurement'] = measurements_df['Measurement'].cat.as_known()

In [15]:
measurements_df.dtypes

Internalpatientid              int32
Age at measurement              int8
Measurement date      datetime64[ns]
Measurement                 category
Result numeric               float64
dtype: object

In [16]:
dummy_df = dd.get_dummies(measurements_df['Measurement'], prefix='Measurement')
measurements_df = measurements_df.merge(dummy_df, left_index=True, right_index=True)
measurements_df = measurements_df.drop(columns=['Measurement'])

In [26]:
measurements_df.dtypes

Internalpatientid                               int32
Age at measurement                               int8
Measurement date                       datetime64[ns]
Result numeric                                float64
Measurement_Central venous pressure             uint8
Measurement_Circumference/girth                 uint8
Measurement_Height                              uint8
Measurement_Pain                                uint8
Measurement_Pulse                               uint8
Measurement_Pulse oximetry                      uint8
Measurement_Respiratory rate                    uint8
Measurement_Temperature                         uint8
Measurement_Weight                              uint8
Measurement_Blood pressure                      uint8
dtype: object

In [10]:
# Save the Dask DataFrame as Parquet
measurements_df.to_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/measurements.parquet', engine='pyarrow')

# Cleaning Lab Results DF

In [6]:
lab_harm_df = pd.read_csv('/content/drive/MyDrive/labHarmonization.csv')
filtered_lab_harm_df = lab_harm_df[lab_harm_df['concept'].notna()].sort_values('sd', ascending=False).drop(columns=['Unnamed: 0']).reset_index(drop=True)
filtered_lab_harm_df

<ipython-input-6-45225c681262>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  lab_harm_df = pd.read_csv('/content/drive/MyDrive/labHarmonization.csv')


,desc,concept,Count,Mean,sd,unit,check,k,na,wbc,...,ddimer,esr,crp,hscrp,meth_sc,meth_lvl,methadone_sc,methadone_lvl,cocaine_sc,cocaine_lvl
0,HIGH SENSITIVITY C-REACTIVE PROTEIN,hscrp,1408,7.630000e+66,2.800000e+68,mg/L,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,COCAINE CONFIRM,cocaine_lvl,14,8.969360e+04,1.980224e+05,ng/mL,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,COCAINE CONFIRMATION GC/MS,cocaine_lvl,2,4.584100e+04,6.405256e+04,ng/mL,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,COCAINE METABOLITES (PB-MA),cocaine_lvl,40,3.155922e+04,5.506092e+04,NG/ML,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,METHAMPHETAMINES -,meth_lvl,4,1.268545e+05,5.386386e+04,ng/mL,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6587,SC POTASSIUM,k,1,4.870000e+00,NaN,mmol/L,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6588,SC CREATININE,cr,1,3.360000e+00,NaN,mg/dL,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6589,TROPONIN I (ISTAT)(DC'D 5-18-17),tropi,1,1.200000e-01,NaN,ng/mL,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6590,D-LACTATE PLASMA,lactate,1,NaN,NaN,NaN,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
labs_to_keep = filtered_lab_harm_df['desc'].tolist()

In [8]:
filtered_lab_harm_df['concept'].value_counts()

bicarb           434
tropi            368
inr              348
cocaine_sc       323
cr               315
hct              293
k                287
hgb              285
gfr              267
a1c              262
ddimer           261
methadone_sc     260
na               260
ph               234
crp              233
wbc              198
pco2             196
alt              191
tbili            167
ldh              159
lactate          153
ferritin         151
esr              144
ast              139
meth_sc          133
bnp              105
methadone_lvl     98
hscrp             80
meth_lvl          67
ntprobnp          63
cocaine_lvl       52
trophs            39
tropt             27
Name: concept, dtype: int64

In [35]:
value_counts = filtered_lab_harm_df['unit'].value_counts()
values_with_count_1 = value_counts[value_counts == 1]
units_to_remove_list = values_with_count_1.index.tolist()

I'm going to remove units with a count of 1. Some of them look like typos and not worth keeping

In [9]:
lab_results_df = dd.read_parquet('/content/drive/MyDrive/VCHAMPS - Train Data -  UTC/lab_results_train.parquet/*.parquet')

In [10]:
lab_results_df.columns

Index(['Internalpatientid', 'Age at lab test', 'Lab test date', 'Lab test',
       'Lab test description', 'Result numeric', 'Result textual',
       'Result range', 'Result units', 'Specimen source'],
      dtype='object')

To simplify things, I'll drop nans from Age at lab test", "Lab test date", "Lab test", "Result numeric",'Result units' and then go from there.

In [11]:
columns_list = ["Age at lab test", "Lab test date", "Lab test", "Result numeric",'Result units']
lab_results_df = lab_results_df.dropna(subset = columns_list)

In [12]:
lab_results_df = lab_results_df.drop(columns=['Result textual','Lab test'])

Now I'll filter the dataframe for the labs that we have a concept for

In [13]:
filtered_lab_results_df = lab_results_df[lab_results_df['Lab test description'].isin(labs_to_keep)]
filtered_lab_results_df = filtered_lab_results_df.reset_index(drop=True)

In [14]:
filtered_lab_results_df.dtypes

Internalpatientid                int32
Age at lab test                   int8
Lab test date           datetime64[ns]
Lab test description            object
Result numeric                 float64
Result range                    object
Result units                    object
Specimen source                 object
dtype: object

In [15]:
dask_df = dd.from_pandas(filtered_lab_harm_df, npartitions=2)

In [16]:
merged_df = dd.merge(filtered_lab_results_df, dask_df, left_on='Lab test description', right_on='desc')

Good the merge worked.

In [17]:
merged_df.dtypes

Internalpatientid                int32
Age at lab test                   int8
Lab test date           datetime64[ns]
Lab test description            object
Result numeric                 float64
Result range                    object
Result units                    object
Specimen source                 object
desc                            object
concept                         object
Count                            int64
Mean                           float64
sd                             float64
unit                            object
check                            int64
k                                int64
na                               int64
wbc                              int64
bicarb                           int64
pco2                             int64
hct                              int64
cr                               int64
hgb                              int64
lactate                          int64
ldh                              int64
ph                       

Need to convert a bunch of these columns to int8

In [18]:
# Assuming dask_df is your Dask DataFrame
columns_to_convert = ['Count', 'Mean', 'sd', 'check', 'k', 'na', 'wbc', 'bicarb', 'pco2', 'hct', 'cr', 'hgb', 'lactate', 'ldh', 'ph', 'gfr', 'ast', 'alt', 'inr', 'a1c', 'ferritin', 'tropi', 'tropt', 'trophs', 'bnp', 'ntprobnp', 'tbili', 'ddimer', 'esr', 'crp', 'hscrp', 'meth_sc', 'meth_lvl', 'methadone_sc', 'methadone_lvl', 'cocaine_sc', 'cocaine_lvl']
for column in columns_to_convert:
    dask_df[column] = dask_df[column].astype('int8')

desc and lab test description are the same. i'll drop desc

I'll round the sd and mean columns to 3 decimal places. We don't need 5-6 digits of precision here

In [19]:
merged_df['Mean'] = merged_df['Mean'].round(3)
merged_df['sd'] = merged_df['sd'].round(3)

I'll drop nans from the Result range columns

In [20]:
merged_df = merged_df.dropna(subset = ['Result units','Result range'])

For now this will do. I'll save it.

In [21]:
split_columns = merged_df['Result range'].str.split(' - ')
merged_df['range_min'] = split_columns.str[0]
merged_df['range_max'] = split_columns.str[1]

# Assuming df is your Dask DataFrame
merged_df['range_min'] = merged_df['range_min'].astype(float)
merged_df['range_max'] = merged_df['range_max'].astype(float)

merged_df = merged_df.drop(columns = ['Result range'])

In [22]:
merged_df = merged_df.drop(columns = ['Lab test description','Result units','check','Count'])

In [23]:
merged_df.dtypes

Internalpatientid             int32
Age at lab test                int8
Lab test date        datetime64[ns]
Result numeric              float64
Specimen source              object
desc                         object
concept                      object
Mean                        float64
sd                          float64
unit                         object
k                             int64
na                            int64
wbc                           int64
bicarb                        int64
pco2                          int64
hct                           int64
cr                            int64
hgb                           int64
lactate                       int64
ldh                           int64
ph                            int64
gfr                           int64
ast                           int64
alt                           int64
inr                           int64
a1c                           int64
ferritin                      int64
tropi                       

In [24]:
columns_to_drop = ['k', 'na', 'wbc', 'bicarb', 'pco2', 'hct', 'cr', 'hgb', 'lactate', 'ldh', 'ph', 'gfr', 'ast', 'alt', 'inr', 'a1c', 'ferritin', 'tropi', 'tropt', 'trophs', 'bnp', 'ntprobnp', 'tbili', 'ddimer', 'esr', 'crp', 'hscrp', 'meth_sc', 'meth_lvl', 'methadone_sc', 'methadone_lvl', 'cocaine_sc', 'cocaine_lvl']
merged_df = merged_df.drop(columns = columns_to_drop)

In [25]:
merged_df.dtypes

Internalpatientid             int32
Age at lab test                int8
Lab test date        datetime64[ns]
Result numeric              float64
Specimen source              object
desc                         object
concept                      object
Mean                        float64
sd                          float64
unit                         object
range_min                   float64
range_max                   float64
dtype: object

In [27]:
merged_df = merged_df.dropna()

Here I'll remove outliers defined by the result numeric being greater or less than mean +/- 2*sd

In [26]:
# Assuming df is your Dask DataFrame and you have columns 'Mean' and 'sd'
merged_df = merged_df[(merged_df['Result numeric'] >= merged_df['Mean'] - 2 * merged_df['sd']) | (merged_df['Result numeric'] <= merged_df['Mean'] + 2 * merged_df['sd'])]

In [29]:
merged_df = merged_df.drop(columns = ['Mean','sd'])

In [36]:
def remove_rows_with_values(df, column_name, values_list):
    filtered_df = df[~df[column_name].isin(values_list)]
    return filtered_df

# Usage example
cleaned_df = remove_rows_with_values(merged_df, 'unit', units_to_remove_list)

I've been having trouble saving this file. I was able to repartition it and after much cleaning it seems to have worked well

In [38]:
cleaned_df = cleaned_df.repartition(npartitions=100)

I'm also converting the concept column to categorical datatype to save some memory. This will also make it easier to get dummies later and makes the df more portable at the moment

In [40]:
cleaned_df['concept'] = cleaned_df['concept'].astype('category')
cleaned_df['concept'] = cleaned_df['concept'].cat.as_known()

In [41]:
cleaned_df.dtypes

Internalpatientid             int32
Age at lab test                int8
Lab test date        datetime64[ns]
Result numeric              float64
Specimen source              object
desc                         object
concept                    category
unit                         object
range_min                   float64
range_max                   float64
dtype: object

In [42]:
# Save the Dask DataFrame as Parquet
cleaned_df.to_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/lab_results.parquet', engine='pyarrow', row_group_size=100000, write_index=False)

# Cleaning Measurements BP DF

In [43]:
measurements_bp_df = dd.read_parquet('/content/drive/MyDrive/VCHAMPS - Train Data -  UTC/measurements_blood_pressure_train.parquet/*.parquet')

In [44]:
measurements_bp_df.columns

Index(['Internalpatientid', 'Age at measurement bp', 'Measurement date',
       'Diastolic bp', 'Systolic bp'],
      dtype='object')

In [46]:
measurements_bp_df = measurements_bp_df.dropna()

In [47]:
measurements_bp_df.dtypes

Internalpatientid                 int32
Age at measurement bp              int8
Measurement date         datetime64[ns]
Diastolic bp                    float64
Systolic bp                     float64
dtype: object

In [48]:
measurements_bp_df = measurements_bp_df.compute()

In [49]:
measurements_bp_df

,Internalpatientid,Age at measurement bp,Measurement date,Diastolic bp,Systolic bp
0,1,62,2005-10-25 00:02:08,75.0,140.0
1,1,67,2011-04-21 02:50:27,72.0,116.0
2,1,68,2012-07-27 03:54:47,100.0,145.0
3,1,68,2012-08-11 22:51:23,89.0,155.0
4,1,69,2013-01-17 16:23:39,72.0,143.0
...,...,...,...,...,...
846502,99999,96,2013-03-28 17:23:42,62.0,147.0
846503,99999,96,2013-03-29 04:17:14,57.0,123.0
846504,99999,96,2013-03-31 04:57:36,68.0,154.0
846505,99999,96,2013-04-09 12:47:57,79.0,147.0


I'm going to convert blood pressure to an integer. It's supposed to be continuous but it is generally recorded as an int. I'll convert it to int16

In [50]:
measurements_bp_df.describe()

,Internalpatientid,Age at measurement bp,Diastolic bp,Systolic bp
count,2.199756e+07,2.199756e+07,2.199756e+07,2.199756e+07
mean,8.521049e+04,6.985868e+01,7.260799e+01,1.318807e+02
std,4.880468e+04,1.145363e+01,1.320071e+01,2.268283e+01
min,1.000000e+00,1.300000e+01,2.900000e+01,4.000000e+01
25%,4.287000e+04,6.200000e+01,6.400000e+01,1.160000e+02
50%,8.590000e+04,7.000000e+01,7.200000e+01,1.310000e+02
75%,1.273410e+05,7.800000e+01,8.100000e+01,1.460000e+02
max,1.690640e+05,1.100000e+02,2.080000e+02,3.120000e+02


min sbp is 40
max sbp is 320
min dbp is 29
max dbp is 208

In [51]:
measurements_bp_df['Diastolic bp'] = measurements_bp_df['Diastolic bp'].astype('int16')
measurements_bp_df['Systolic bp'] = measurements_bp_df['Systolic bp'].astype('int16')

In [52]:
measurements_bp_df.dtypes

Internalpatientid                 int32
Age at measurement bp              int8
Measurement date         datetime64[ns]
Diastolic bp                      int16
Systolic bp                       int16
dtype: object

In [54]:
nan_percentage = calculate_nan_percentage(measurements_bp_df)
nan_percentage

Internalpatientid        0.0
Age at measurement bp    0.0
Measurement date         0.0
Diastolic bp             0.0
Systolic bp              0.0
dtype: float64

No more NaNs. I think that's all that I can do for now.

In [55]:
# Save the Dask DataFrame as Parquet
measurements_bp_df.to_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/measurements_bp.parquet', engine='pyarrow')

# Cleaning Medication Administered DF

In [7]:
medications_administered_df = dd.read_parquet('/content/drive/MyDrive/VCHAMPS - Train Data -  UTC/medications_administered_train.parquet/*.parquet')

In [8]:
medications_administered_df.columns

Index(['Internalpatientid', 'Age at med administration', 'Administration date',
       'Administration end date', 'Administered medication atc 5',
       'Administration status', 'Dose form', 'Dose administered',
       'Dose unit administered'],
      dtype='object')

In [9]:
medications_administered_df.dtypes

Internalpatientid                         int32
Age at med administration                  int8
Administration date              datetime64[ns]
Administration end date          datetime64[ns]
Administered medication atc 5            object
Administration status                    object
Dose form                                object
Dose administered                       float64
Dose unit administered                   object
dtype: object

In [10]:
medications_administered_df = medications_administered_df.compute()

In [60]:
medications_administered_df

,Internalpatientid,Age at med administration,Administration date,Administration end date,Administered medication atc 5,Administration status,Dose form,Dose administered,Dose unit administered
0,1,68,2011-12-12 15:45:21,NaT,omega-3-triglycerides,Given,"cap,oral",3.0,"cap,oral"
1,1,68,2011-12-12 23:26:46,NaT,metoprolol,Given,tab,3.0,tab
2,1,68,2012-08-12 19:19:33,NaT,omega-3-triglycerides,Given,"cap,oral",3.0,"cap,oral"
3,1,68,2012-08-14 03:13:09,NaT,metoprolol,Given,tab,1.0,tab
4,1,72,2016-07-19 06:57:34,NaT,metoprolol,Given,tab,1.0,tab
...,...,...,...,...,...,...,...,...,...
593084,99999,96,2013-04-10 01:14:15,NaT,salbutamol,Given,"soln,inhl",1.0,1 amp
593085,99999,96,2013-04-11 01:09:47,NaT,insulin (human) fast-acting,Held,inj,0.0,None
593086,99999,96,2013-04-12 05:22:34,NaT,Not specified,Given,inj,1.0,90ml
593087,99999,96,2013-04-14 02:00:34,NaT,docusate,Held,"cap,oral",0.0,"cap,oral"


In [11]:
medications_administered_df['Administration status'].value_counts()

Given                          111703927
Held                             6139859
Refused                          3470929
Completed (infusion orders)      1231777
Removed (patch taken off)         421904
Missing dose                      384054
Infusing (infusion orders)        381964
Not given                          75922
Not specified (no value)           31599
Stopped (infusion orders)           7650
Name: Administration status, dtype: int64

Remove Not specified

In [12]:
medications_administered_df = medications_administered_df[~(medications_administered_df['Administration status'] == 'Not specified (no value)')]

Let me look at Dose form

In [13]:
dose_form_list = medications_administered_df['Dose form'].value_counts()

In [75]:
dose_form_list

tab               49254843
inj               13533946
cap,oral           8880813
inj,soln           6448691
soln,inhl          5120018
                    ...   
varnish,dental           1
powder,rtl               1
implant                  1
tab/supp                 1
tab,buccal               1
Name: Dose form, Length: 229, dtype: int64

Hmmm, miscellaneous? I'll remove that since its undescriptive
Remove *unknown at this time*     33



In [14]:
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose form'] == 'miscellaneous') | (medications_administered_df['Dose form'] == '*unknown at this time*'))]

Let me see the NaN situation

In [77]:
nan_percentage = calculate_nan_percentage(medications_administered_df)
nan_percentage

Internalpatientid                 0.000000
Age at med administration         0.000000
Administration date               0.000000
Administration end date          98.656412
Administered medication atc 5     0.000000
Administration status             0.000000
Dose form                         0.000000
Dose administered                 2.849982
Dose unit administered            6.215448
dtype: float64

I'll just drop the NaNs in Dose administered and Dose unit administered

In [15]:
medications_administered_df = medications_administered_df.dropna(subset = ['Dose administered','Dose unit administered'])

In [79]:
medications_administered_df

,Internalpatientid,Age at med administration,Administration date,Administration end date,Administered medication atc 5,Administration status,Dose form,Dose administered,Dose unit administered
0,1,68,2011-12-12 15:45:21,NaT,omega-3-triglycerides,Given,"cap,oral",3.0,"cap,oral"
1,1,68,2011-12-12 23:26:46,NaT,metoprolol,Given,tab,3.0,tab
2,1,68,2012-08-12 19:19:33,NaT,omega-3-triglycerides,Given,"cap,oral",3.0,"cap,oral"
3,1,68,2012-08-14 03:13:09,NaT,metoprolol,Given,tab,1.0,tab
4,1,72,2016-07-19 06:57:34,NaT,metoprolol,Given,tab,1.0,tab
...,...,...,...,...,...,...,...,...,...
593083,99999,96,2013-04-09 15:00:44,NaT,docusate,Given,"cap,oral",1.0,"cap,oral"
593084,99999,96,2013-04-10 01:14:15,NaT,salbutamol,Given,"soln,inhl",1.0,1 amp
593086,99999,96,2013-04-12 05:22:34,NaT,Not specified,Given,inj,1.0,90ml
593087,99999,96,2013-04-14 02:00:34,NaT,docusate,Held,"cap,oral",0.0,"cap,oral"


In [19]:
medications_administered_df = medications_administered_df[~((medications_administered_df['Administered medication atc 5'] == '(Censored)'))]

In [20]:
medications_administered_df = medications_administered_df[~((medications_administered_df['Administered medication atc 5'] == 'Not specified'))]

In [21]:
medications_administered_df

,Internalpatientid,Age at med administration,Administration date,Administration end date,Administered medication atc 5,Administration status,Dose form,Dose administered,Dose unit administered
0,1,68,2011-12-12 15:45:21,NaT,omega-3-triglycerides,Given,"cap,oral",3.0,"cap,oral"
1,1,68,2011-12-12 23:26:46,NaT,metoprolol,Given,tab,3.0,tab
2,1,68,2012-08-12 19:19:33,NaT,omega-3-triglycerides,Given,"cap,oral",3.0,"cap,oral"
3,1,68,2012-08-14 03:13:09,NaT,metoprolol,Given,tab,1.0,tab
4,1,72,2016-07-19 06:57:34,NaT,metoprolol,Given,tab,1.0,tab
...,...,...,...,...,...,...,...,...,...
593082,99999,96,2013-04-09 04:06:23,NaT,insulin (human) fast-acting,Given,inj,1.0,2 units
593083,99999,96,2013-04-09 15:00:44,NaT,docusate,Given,"cap,oral",1.0,"cap,oral"
593084,99999,96,2013-04-10 01:14:15,NaT,salbutamol,Given,"soln,inhl",1.0,1 amp
593087,99999,96,2013-04-14 02:00:34,NaT,docusate,Held,"cap,oral",0.0,"cap,oral"


In [73]:
grouped_data = medications_administered_df.groupby('Administered medication atc 5').agg({'Dose administered': 'mean',
                                                                                         'Administered medication atc 5': 'count',
                                                                                         'Dose unit administered': 'first',
                                                                                         'Dose form': lambda x: x.mode().iat[0]
                                                                                         })

In [74]:
grouped_data

,Dose administered,Administered medication atc 5,Dose unit administered,Dose form
Administered medication atc 5,,,,
Not a medication,31.853536,371702,thin layer,test strip
abacavir,1.256728,3716,tab,tab
abacavir ; dolutegravir,0.994536,732,tab,tab
abacavir ; dolutegravir ; lamivudine,1.000000,424,tab,tab
abacavir ; lamivudine,0.964346,2356,tab,tab
...,...,...,...,...
zolmitriptan,0.891903,657,"tab,oral disintegrating",tab
zolpidem,1.117816,124074,tab,tab
zonisamide,2.287430,2331,"cap,oral","cap,oral"


In [75]:
grouped_data.to_csv('/content/drive/MyDrive/medications_administered_grouped_data.csv', index=True)

In [76]:
magd = pd.read_csv('/content/drive/MyDrive/medications_administered_grouped_data.csv')
magd

,Administered medication atc 5,Dose administered,Administered medication atc 5.1,Dose unit administered,Dose form
0,Not a medication,31.853536,371702,thin layer,test strip
1,abacavir,1.256728,3716,tab,tab
2,abacavir ; dolutegravir,0.994536,732,tab,tab
3,abacavir ; dolutegravir ; lamivudine,1.000000,424,tab,tab
4,abacavir ; lamivudine,0.964346,2356,tab,tab
...,...,...,...,...,...
2520,zolmitriptan,0.891903,657,"tab,oral disintegrating",tab
2521,zolpidem,1.117816,124074,tab,tab
2522,zonisamide,2.287430,2331,"cap,oral","cap,oral"
2523,"zoster, live attenuated",1.000000,230,0.65ml,"inj,lyphl"


In [24]:
dose_unit_admin_list = medications_administered_df['Dose unit administered'].value_counts()

In [40]:
dose_unit_admin_list[600:]

tab (delayed release)    4891
3 mg                     4859
35u                      4851
1 tablespoonful          4850
2g                       4821
                         ... 
3 units for 147             1
0.1 ml right arm            1
44096                       1
5000 unit//verified         1
2.5mg inhaled               1
Name: Dose unit administered, Length: 311680, dtype: int64

Below I'll remove entries with vague or uninformative Dose Unit administered

In [34]:
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == '.'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'as ordered'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'moderate amount'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'small amt'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'mod amt'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'as directed'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'ad'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'liberal amount'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'moderate'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'per order'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'liberally'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'ok'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'ao'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'lib amt'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'liberal'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'sm amount'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'a small amount'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'suff'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'some'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'as order'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'as ord'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'a'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'ade'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'dose'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'as rx'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'mod. amt.'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'suff amount'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'smamt'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'as'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'p'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'small amnt'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'adequate amount'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'moderate amt.'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'sm amnt'))]
medications_administered_df = medications_administered_df[~((medications_administered_df['Dose unit administered'] == 'aso'))]

I'll also convert Administration status to a categorical variable

In [43]:
medications_administered_df['Administration status'].value_counts()

Given                          101681349
Held                             1613816
Refused                           767616
Completed (infusion orders)       656683
Removed (patch taken off)         420943
Infusing (infusion orders)        175563
Missing dose                      104254
Not given                          68957
Stopped (infusion orders)            816
Name: Administration status, dtype: int64

In [46]:
medications_administered_df['Administration status'] = medications_administered_df['Administration status'].astype('category')

In [47]:
medications_administered_df.dtypes

Internalpatientid                         int32
Age at med administration                  int8
Administration date              datetime64[ns]
Administration end date          datetime64[ns]
Administered medication atc 5            object
Administration status                  category
Dose form                                object
Dose administered                       float64
Dose unit administered                   object
dtype: object

In [48]:
medications_administered_df

,Internalpatientid,Age at med administration,Administration date,Administration end date,Administered medication atc 5,Administration status,Dose form,Dose administered,Dose unit administered
0,1,68,2011-12-12 15:45:21,NaT,omega-3-triglycerides,Given,"cap,oral",3.0,"cap,oral"
1,1,68,2011-12-12 23:26:46,NaT,metoprolol,Given,tab,3.0,tab
2,1,68,2012-08-12 19:19:33,NaT,omega-3-triglycerides,Given,"cap,oral",3.0,"cap,oral"
3,1,68,2012-08-14 03:13:09,NaT,metoprolol,Given,tab,1.0,tab
4,1,72,2016-07-19 06:57:34,NaT,metoprolol,Given,tab,1.0,tab
...,...,...,...,...,...,...,...,...,...
593082,99999,96,2013-04-09 04:06:23,NaT,insulin (human) fast-acting,Given,inj,1.0,2 units
593083,99999,96,2013-04-09 15:00:44,NaT,docusate,Given,"cap,oral",1.0,"cap,oral"
593084,99999,96,2013-04-10 01:14:15,NaT,salbutamol,Given,"soln,inhl",1.0,1 amp
593087,99999,96,2013-04-14 02:00:34,NaT,docusate,Held,"cap,oral",0.0,"cap,oral"


In [49]:
medications_administered_ddf  = dd.from_pandas(medications_administered_df, npartitions=100)

In [50]:
# Save the Dask DataFrame as Parquet
medications_administered_ddf.to_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/medications_administered.parquet', engine='pyarrow')

# Cleaning Medication Ordered DF

In [51]:
medications_ordered_df = dd.read_parquet('/content/drive/MyDrive/VCHAMPS - Train Data -  UTC/medications_ordered_train.parquet/*.parquet')

In [55]:
medications_ordered_df.columns

Index(['Internalpatientid', 'Age at med ordered', 'Order date',
       'Medication start date', 'Medication stop date',
       'Order discontinue date', 'Ordered medication atc 5', 'Order status',
       'Stop reason'],
      dtype='object')

In [56]:
medications_ordered_df.dtypes

Internalpatientid                    int32
Age at med ordered                    int8
Order date                  datetime64[ns]
Medication start date       datetime64[ns]
Medication stop date        datetime64[ns]
Order discontinue date      datetime64[ns]
Ordered medication atc 5            object
Order status                        object
Stop reason                         object
dtype: object

In [58]:
medications_ordered_df = medications_ordered_df.compute()

In [59]:
medications_ordered_df

,Internalpatientid,Age at med ordered,Order date,Medication start date,Medication stop date,Order discontinue date,Ordered medication atc 5,Order status,Stop reason
0,1,60,2004-01-09 14:07:46,2004-01-09 01:33:46,2004-04-02 11:27:46,2004-04-02 11:27:46,lisinopril,discontinued/edit,Not specified (no value)
1,1,60,2004-04-04 14:19:12,2004-04-04 04:26:12,2005-03-07 04:26:12,2005-04-17 15:19:12,Not specified,discontinued,Not specified (no value)
2,1,61,2005-01-20 08:43:00,2005-01-19 20:06:00,2005-06-29 05:18:00,2005-06-29 05:18:00,lisinopril,discontinued,Requesting Physician Cancelled
3,1,61,2005-05-25 10:56:29,2005-05-24 20:23:29,2005-06-23 20:23:29,NaT,amoxicillin,expired,Not specified (no value)
4,1,63,2007-07-09 01:43:28,2007-07-08 20:21:28,2007-08-07 20:21:28,NaT,Not specified,expired,Not specified (no value)
...,...,...,...,...,...,...,...,...,...
364651,99999,96,2013-03-21 16:04:43,2013-03-21 16:03:43,2013-03-22 16:03:43,NaT,docusate,expired,Not specified (no value)
364652,99999,96,2013-03-21 16:04:43,2013-03-21 16:04:43,2013-03-22 16:04:43,NaT,citalopram,expired,Not specified (no value)
364653,99999,96,2013-04-04 23:20:47,2013-04-04 23:30:47,2013-04-10 04:24:47,2013-04-10 04:24:47,magnesium hydroxide,discontinued,Not specified (no value)
364654,99999,96,2013-04-08 03:25:27,2013-04-08 03:26:27,2013-04-09 23:43:27,2013-04-09 23:43:27,haloperidol,discontinued/edit,Not specified (no value)


In [60]:
medications_ordered_df['Order status'].value_counts()

discontinued         34632224
expired              14205378
discontinued/edit     6327081
cancelled             2157038
complete               997778
active                 578254
lapsed                 125979
unreleased               8175
hold                     4890
pending                  4567
delayed                  1353
renewed                   662
Name: Order status, dtype: int64

Convert to categorical

In [61]:
medications_ordered_df['Order status'] = medications_ordered_df['Order status'].astype('category')

I'll remove medications that have been censored or not specified

In [62]:
medications_ordered_df = medications_ordered_df[~((medications_ordered_df['Ordered medication atc 5'] == '(Censored)'))]
medications_ordered_df = medications_ordered_df[~((medications_ordered_df['Ordered medication atc 5'] == 'Not specified'))]

In [77]:
stop_reason_list = medications_ordered_df['Stop reason'].value_counts()

I'll wait on Jeff to clean this up further.

In [80]:
medications_ordered_ddf = dd.from_pandas(medications_ordered_df, npartitions=100)

In [81]:
medications_ordered_ddf.dtypes

Internalpatientid                    int32
Age at med ordered                    int8
Order date                  datetime64[ns]
Medication start date       datetime64[ns]
Medication stop date        datetime64[ns]
Order discontinue date      datetime64[ns]
Ordered medication atc 5            object
Order status                      category
Stop reason                         object
dtype: object

In [82]:
# Save the Dask DataFrame as Parquet
medications_ordered_ddf.to_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/medications_ordered.parquet', engine='pyarrow')

# Cleaning outpatient visits

In [6]:
outpatient_visits_df = dd.read_parquet('/content/drive/MyDrive/VCHAMPS - Train Data -  UTC/outpatient_visits_train.parquet/*.parquet')

In [7]:
outpatient_visits_df.columns

Index(['Internalpatientid', 'Age at visit', 'Visit start date',
       'Visit End Date', 'First listed diagnosis icd10 subcategory',
       'Second listed diagnosis icd10 subcategory', 'Stop code',
       'Agentorangeflag', 'Combatflag', 'Ionizingradiationflag',
       'Serviceconnectedflag', 'Swasiaconditionsflag'],
      dtype='object')

In [8]:
outpatient_visits_df.dtypes

Internalpatientid                                     int32
Age at visit                                           int8
Visit start date                             datetime64[ns]
Visit End Date                               datetime64[ns]
First listed diagnosis icd10 subcategory             object
Second listed diagnosis icd10 subcategory            object
Stop code                                            object
Agentorangeflag                                      object
Combatflag                                           object
Ionizingradiationflag                                object
Serviceconnectedflag                                 object
Swasiaconditionsflag                                 object
dtype: object

I know that AgentOrange,combatflag, ionizing, serviceconnected, and swasiaconditions should be categorical variables so I'll make them so

In [9]:
cols_to_convert = ['Agentorangeflag','Combatflag','Ionizingradiationflag','Serviceconnectedflag','Swasiaconditionsflag']
for col in cols_to_convert:
  outpatient_visits_df[col] = outpatient_visits_df[col].astype('category')
  outpatient_visits_df[col] = outpatient_visits_df[col].cat.as_known()

In [10]:
outpatient_visits_df.dtypes

Internalpatientid                                     int32
Age at visit                                           int8
Visit start date                             datetime64[ns]
Visit End Date                               datetime64[ns]
First listed diagnosis icd10 subcategory             object
Second listed diagnosis icd10 subcategory            object
Stop code                                            object
Agentorangeflag                                    category
Combatflag                                         category
Ionizingradiationflag                              category
Serviceconnectedflag                               category
Swasiaconditionsflag                               category
dtype: object

Now I'll do the ICD mapping

In [ ]:
outpatient_visits_df = process_diagnoses_dataframe(outpatient_visits_df, icd_codes)

I'll drop nans from code and diagnosis columns

In [15]:
outpatient_visits_df = outpatient_visits_df.drop(columns = ['billable'])
outpatient_visits_df = outpatient_visits_df.dropna(subset = ['code','diagnosis'])

In [16]:
outpatient_visits_df.dtypes

Internalpatientid                 int32
Age at visit                       int8
Visit start date         datetime64[ns]
Visit End Date           datetime64[ns]
Stop code                        object
Agentorangeflag                category
Combatflag                     category
Ionizingradiationflag          category
Serviceconnectedflag           category
Swasiaconditionsflag           category
diagnosis                        object
code                             object
cc Status                        object
dtype: object

In [ ]:
outpatient_visits_df = outpatient_visits_df.compute()